In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [26]:
training_set = pd.read_csv(r"C:\Users\risha\Desktop\Assignment\train_data - Sheet1.csv")
training_imgs = [x for x in list(training_set.Images)]
training_labels_1 = list(training_set['Name'])
training_labels_2 = list(training_set['Weapon_name'])
training_labels_3 = list(training_set['Action'])
training_labels_4 = list(training_set['Intent'])
training_set = pd.DataFrame( {'Images': training_imgs,'Name': training_labels_1, 'Weapon_name' : training_labels_2, 'Action' : training_labels_3, 'Intent' : training_labels_4})
#Changing the type  to str
training_set.Name = training_set.Name.astype(str)
training_set.Weapon_name = training_set.Weapon_name.astype(str)
training_set.Action = training_set.Action.astype(str)
training_set.Intent = training_set.Intent.astype(str)

In [27]:
training_set['New_class'] = training_set['Name'] + training_set['Weapon_name'] + training_set['Action'] + training_set['Intent']

In [28]:
print(training_set.head())

   Images    Name Weapon_name    Action Intent                 New_class
0  1.jpeg   Human       Sword  Standing  NoWar   HumanSwordStandingNoWar
1  2.jpeg   Human      Chakra  Standing  NoWar  HumanChakraStandingNoWar
2  3.jpeg   Human     Dhanush  Standing    War   HumanDhanushStandingWar
3  4.jpeg   Human      Chakra   Running    War     HumanChakraRunningWar
4  5.jpeg  Animal     Nothing  Fighting    War  AnimalNothingFightingWar


In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)
train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="",x_col="Images",
                                        y_col="New_class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=32)

Found 5 validated image filenames belonging to 5 classes.


In [31]:
classifier = Sequential()
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Flatten())
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dense(units = 5 , activation = 'softmax'))

In [32]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])

In [33]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 56)      1568      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 56)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 32)        16160     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 28800)             0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [9]:
classifier.fit_generator(train_generator, epochs = 5)

Epoch 1/5


C:\Users\risha\AppData\Local\Temp/ipykernel_18524/92086068.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(train_generator, epochs = 5)


1/1 [==============================] - 0s 477ms/step - loss: 1.6428 - categorical_accuracy: 0.2000 - accuracy: 0.2000
Epoch 2/5
1/1 [==============================] - 0s 123ms/step - loss: 1.8320 - categorical_accuracy: 0.4000 - accuracy: 0.4000
Epoch 3/5
1/1 [==============================] - 0s 119ms/step - loss: 1.5844 - categorical_accuracy: 0.4000 - accuracy: 0.4000
Epoch 4/5
1/1 [==============================] - 0s 118ms/step - loss: 1.3095 - categorical_accuracy: 0.4000 - accuracy: 0.4000
Epoch 5/5
1/1 [==============================] - 0s 120ms/step - loss: 1.0380 - categorical_accuracy: 0.8000 - accuracy: 0.8000


In [10]:
test_set = pd.read_csv(r"C:\Users\risha\Desktop\Assignment\test_data.csv")
test_imgs = [x for x in list(test_set.Images)]
test_set = pd.DataFrame( {'Images': test_imgs })

In [11]:
classes = train_generator.class_indices
print(classes)

{'AnimalNothingFightingWar': 0, 'HumanChakraRunningWar': 1, 'HumanChakraStandingNoWar': 2, 'HumanDhanushStandingWar': 3, 'HumanSwordStandingNoWar': 4}


In [13]:
inverted_classes = dict(map(reversed, classes.items()))
print(inverted_classes)

{0: 'AnimalNothingFightingWar', 1: 'HumanChakraRunningWar', 2: 'HumanChakraStandingNoWar', 3: 'HumanDhanushStandingWar', 4: 'HumanSwordStandingNoWar'}


In [18]:
from keras.preprocessing import image
Y_pred = []
for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = np.argmax(classifier.predict(test_img), axis=-1)
  prediction = img_class[0]
  Y_pred.append(prediction)

In [19]:
print(Y_pred)

[4, 2, 3, 1, 0]


In [20]:
prediction_classes = [ inverted_classes.get(item,item) for item in Y_pred ]

In [21]:
print(prediction_classes)

['HumanSwordStandingNoWar', 'HumanChakraStandingNoWar', 'HumanDhanushStandingWar', 'HumanChakraRunningWar', 'AnimalNothingFightingWar']


Prediction on a single Image

In [24]:
from keras.preprocessing import image
Y_pred=[]
img=image.load_img(path=r"C:\Users\risha\Desktop\Assignment\test\192-1925014_arjun-mahabharat-3d-animation-movie-arjun-mahabharat.jpg",target_size=(128,128,3))
img=image.img_to_array(img)
test_img=img.reshape((1,128,128,3))
img_class=np.argmax(classifier.predict(test_img),axis=-1)
prediction=img_class[0]
Y_pred.append(prediction)
prediction_classes=[inverted_classes.get(item,item) for item in Y_pred]
print(prediction_classes)

['HumanSwordStandingNoWar']
